In [99]:
import pandas as pd

movies = pd.read_csv('data/movies.csv')

In [100]:
rows = []

for _, row in movies.iterrows():
    for genre in row['genres'].split('|'):
        rows.append([genre, row['movieId']])
        
movies_genres = pd.DataFrame(rows, columns=['genre', 'movieId'])
movies_genres['genre_id'] = movies_genres['genre'].astype('category').cat.codes.copy() # сопостовление какому Id фильма, соответствует Id жанра
movies_genres.head()

,genre,movieId,genre_id
0,Animation,1,2
1,Children's,1,3
2,Comedy,1,4
3,Adventure,2,1
4,Children's,2,3


In [101]:
# поскольку строится гибридная система, оценки тоже нужны

ratings = pd.read_csv('data/ratings.csv')

In [102]:
from scipy.sparse import coo_matrix
import numpy as np

user_item_matrix = coo_matrix((
    (ratings["rating"]>=4).astype(np.float32), # по колонке оценок пораждается булевская колонка "нравится"
    (ratings["userId"], ratings["movieId"])),    # назначение матрицы строк и столбцов
    shape=(
        ratings["userId"].unique().max() + 1,   # явно указан размер, так как есть фильмы без оценок, но с жанрами
        movies["movieId"].unique().max() + 1)   # чтобы алгоритм не сломался
)

user_item_matrix.eliminate_zeros()

In [103]:
# делим разреженную матрицу на обучающую и тестовую
total_len = user_item_matrix.data.size
train_len = int(total_len * 0.8)
all_indices = np.arange(total_len)
np.random.seed(42)
train_indices = np.random.choice(all_indices, train_len, replace=False)
train_mask = np.in1d(all_indices, train_indices)

In [104]:
# функция наложения маски на разреженные матрицы
def get_masked(arr, mask):
    return coo_matrix(
        (
            [np.float32(item) for item in arr.data[mask]],
            (arr.row[mask], arr.col[mask])
        ), arr.shape
    )

In [105]:
# подготовка train и  test матриц

train = get_masked(user_item_matrix, train_mask)
test = get_masked(user_item_matrix, ~train_mask)

In [106]:
# сохранения train и test  в npz

from scipy.sparse import save_npz

save_npz('data/lightfm_train.npz', train)
save_npz('data/lightfm_test.npz', test)

Установка   
!conda install -c conda-forge lightfm

##### Подключаем модель

In [107]:
from lightfm import LightFM

fm = LightFM(loss='warp', max_sampled=15)

In [108]:
# обучение модели

fm.fit(interactions=train, 
       epochs=100, 
       num_threads=6, # количество потоков 
       verbose=True)

Epoch: 100%|██████████| 100/100 [00:56<00:00,  1.76it/s]


In [109]:
fm.get_params() # параметры обученной модели

{'loss': 'warp',
 'learning_schedule': 'adagrad',
 'no_components': 10,
 'learning_rate': 0.05,
 'k': 5,
 'n': 10,
 'rho': 0.95,
 'epsilon': 1e-06,
 'max_sampled': 15,
 'item_alpha': 0.0,
 'user_alpha': 0.0,
 'random_state': RandomState(MT19937) at 0x7FD6AABCF490}

In [110]:
%%time

from lightfm.evaluation import reciprocal_rank

rr = reciprocal_rank(model=fm, test_interactions=test, train_interactions=train, num_threads=6) # дисконтированный хитрейт

CPU times: user 1.89 s, sys: 7.24 ms, total: 1.9 s
Wall time: 1.92 s


In [111]:
rr.mean() # среднее по всем пользователям

0.49834794

In [112]:
%%time

fm.fit_partial(interactions=train, epochs=100, num_threads=6, verbose=True) # продолжение обучения с прерванного момента

Epoch: 100%|██████████| 100/100 [00:57<00:00,  1.75it/s]

CPU times: user 55.7 s, sys: 336 ms, total: 56.1 s
Wall time: 57 s


In [113]:
%%time

rr = reciprocal_rank(model=fm, test_interactions=test, train_interactions=train, num_threads=6)

CPU times: user 1.9 s, sys: 6.69 ms, total: 1.91 s
Wall time: 1.91 s


In [114]:
rr.mean()

0.50210655

In [115]:
user_factors = fm.get_user_representations()
print(user_factors)

(array([  0.       ,  -6.6424117,  -8.6195345, ...,  -4.14148  ,
        -8.533468 , -14.989444 ], dtype=float32), array([[-0.02183516,  0.03104008,  0.0339275 , ..., -0.00456486,
        -0.0218044 , -0.00847927],
       [ 0.66724837,  0.00184182, -0.36907074, ..., -0.814388  ,
         0.4633758 ,  0.7133973 ],
       [ 0.12351669, -0.25644386,  1.0144219 , ..., -0.15825948,
         0.64981884, -0.17695291],
       ...,
       [ 0.36860806,  0.7782866 , -0.09230369, ..., -0.64138794,
        -0.17150559, -0.23478888],
       [ 0.24513043,  1.018095  , -0.56260115, ..., -1.4754514 ,
         0.2233494 ,  0.21332413],
       [-0.6218216 ,  1.2399174 ,  0.02535371, ..., -0.2644478 ,
         0.5981313 ,  1.0441839 ]], dtype=float32))


In [116]:
print(len(user_factors[0]))
print(user_factors[1].shape) # размерность матрицы факторов пользователя = пользователи на скрытые факторы

6041
(6041, 10)


In [117]:
item_factors = fm.get_item_representations() # тоже самое с факторами объектов
print(item_factors)

(array([-1.4959909 ,  2.2405884 ,  0.7839686 , ..., -0.45476004,
       -0.64014304,  0.6948971 ], dtype=float32), array([[ 0.6542956 ,  0.3156788 ,  0.3252783 , ...,  0.5602816 ,
        -0.87492496, -0.39728847],
       [ 0.35843417, -0.34476957, -0.8943662 , ..., -0.5662831 ,
         0.562454  ,  0.14841406],
       [ 0.45888364, -0.34363794, -1.1960322 , ...,  0.12966253,
        -0.2421121 , -0.7184711 ],
       ...,
       [ 0.37442115, -0.46336985,  0.36944586, ...,  0.9114857 ,
         0.14617042,  1.1479101 ],
       [ 0.7472734 ,  0.7029012 ,  1.2749847 , ...,  0.3891351 ,
        -0.1339105 ,  0.83127147],
       [ 0.8097521 ,  0.05532259,  0.857232  , ...,  1.2192491 ,
         1.2410944 ,  0.5006391 ]], dtype=float32))


In [118]:
print(len(item_factors[0]))
print(item_factors[1].shape) # размерность матрицы факторов item = пользователи на скрытые факторы

3953
(3953, 10)


### Добавление фичей жанров и id жанров

In [119]:
# добавление фичей жанров и id жанров

from scipy.sparse import identity, hstack

item_feature_matrix = hstack([  # контектенирует две матрицы
    coo_matrix(
        (np.ones(movies_genres.count()[0], dtype=np.float32),
        (movies_genres['movieId'], movies_genres['genre_id'])),
        shape=(user_item_matrix.shape[1], movies_genres['genre_id'].unique().size)),
        identity(user_item_matrix.shape[1]) # единичная матрица соответствия жанра
])

item_feature_matrix.shape # результат = id фильма + id жанров

(3953, 3971)

In [120]:
%%time

fm.fit(interactions=train, epochs=100, item_features=item_feature_matrix, verbose=True) # обучается так же плюс передается матрица фичей

Epoch: 100%|██████████| 100/100 [01:17<00:00,  1.29it/s]

CPU times: user 1min 16s, sys: 428 ms, total: 1min 17s
Wall time: 1min 17s


In [121]:
new_item_factors = fm.get_item_representations() # тоже самое с факторами объектов
print(new_item_factors)


(array([ 0.6360203 ,  0.23552993,  0.6360543 , ..., -0.18580446,
       -0.7905941 ,  0.7993212 ], dtype=float32), array([[-0.02897846,  0.5457959 ,  0.0607111 , ...,  0.6498681 ,
         0.32078242, -0.16437127],
       [ 0.0515788 ,  0.1392302 , -0.4173382 , ..., -0.07515994,
        -0.24397483,  0.11735915],
       [-0.50855505, -0.03343564,  0.12565404, ...,  0.18378513,
        -0.7751745 ,  0.21144849],
       ...,
       [ 0.06902822,  0.1241332 ,  0.36874452, ...,  0.14713833,
         0.24951597,  0.5650499 ],
       [-0.2827888 , -0.5775592 ,  1.3828031 , ...,  0.33989358,
         0.01571707, -0.24833426],
       [ 0.2623806 , -0.7226788 ,  0.46746948, ...,  0.6482909 ,
         0.16311009, -0.7373382 ]], dtype=float32))


In [122]:
print(len(new_item_factors[0]))
print(new_item_factors[1].shape)

3971
(3971, 10)


In [123]:
%%time
# оценка

rr = reciprocal_rank(model=fm, test_interactions=test, train_interactions=train, 
                     item_features=item_feature_matrix, num_threads=6)

CPU times: user 2.35 s, sys: 19.6 ms, total: 2.37 s
Wall time: 2.47 s


In [124]:
rr.mean()

0.5069311

In [125]:
# как это не печально, но результат получился гораздо хуже.

#### Задание слушателям  
1. Подобрать параметры модели и улучшить результат.